In [1]:
import pickle
import os
import torch
import numpy as np
import IPython.display as ipd
from util.evaluate import Evaluator
from factory.AutoVC_Adjust import AutoVC_Adjust

In [2]:
class Config:
    def __init__(self,root):
        self.root = root
        self.num_speaker = 5
        self.batch_size = 2
        self.erroment_uttr_idx = 16
        self.max_uttr_idx = 60
        self.len_crop = 176
        self.device = "cuda:0"
        self.judge = torch.load('model/static/metadv_en_openslr.pt').to(self.device)
        self.all_speaker = sorted(next(iter(os.walk(root)))[1][:self.num_speaker])
        self.metadata = pickle.load(open(f'{root}/train.pkl', "rb"))[:self.num_speaker]

In [3]:
config = Config('train_data')

In [4]:
org_ad = AutoVC_Adjust(44,256,512,22).to(config.device)
org_ad.load_state_dict(torch.load("model/autovc_adjust_en.pt", map_location=config.device))

<All keys matched successfully>

In [5]:
E = Evaluator(config)

Detect Judge ! generate all Real Data d-vector
Processing --- ID:0 Speaker:p225 ---
Processing --- ID:1 Speaker:p226 ---
Processing --- ID:2 Speaker:p227 ---
Processing --- ID:3 Speaker:p228 ---
Processing --- ID:4 Speaker:p229 ---


In [6]:
real_cos_result = E.get_real_data_result()

Processing --- ID:0 Speaker:p225 ---
Processing --- ID:1 Speaker:p226 ---
Processing --- ID:2 Speaker:p227 ---
Processing --- ID:3 Speaker:p228 ---
Processing --- ID:4 Speaker:p229 ---


In [7]:
np.diagonal(real_cos_result).mean(), (np.sum(real_cos_result) - (np.sum(np.diagonal(real_cos_result)))) /(config.num_speaker*(config.num_speaker-1))

(0.9059524297714233, 0.6535484880208969)

In [8]:
all_cos_result = E.generate_result([org_ad],True)

Now Processing --- p225
Reconstruct ---- p225 to p225
Trans --- p225 to p226
Trans --- p225 to p227
Trans --- p225 to p228
Trans --- p225 to p229
Now Processing --- p226
Trans --- p226 to p225
Reconstruct ---- p226 to p226
Trans --- p226 to p227
Trans --- p226 to p228
Trans --- p226 to p229
Now Processing --- p227
Trans --- p227 to p225
Trans --- p227 to p226
Reconstruct ---- p227 to p227
Trans --- p227 to p228
Trans --- p227 to p229
Now Processing --- p228
Trans --- p228 to p225
Trans --- p228 to p226
Trans --- p228 to p227
Reconstruct ---- p228 to p228
Trans --- p228 to p229
Now Processing --- p229
Trans --- p229 to p225
Trans --- p229 to p226
Trans --- p229 to p227
Trans --- p229 to p228
Reconstruct ---- p229 to p229


In [10]:
E.get_cos_rc(all_cos_result[0])

(0.9357142686843872, 0.7151624381542205)

In [12]:
E.get_cos_trans(all_cos_result[0])

(0.8844038188457489, 0.759699838757515)